In [64]:
from tree_sitter import Language, Parser
import  tree_sitter_java as tsjava
JAVA_LANGUAGE = Language(tsjava.language())
parser = Parser(JAVA_LANGUAGE)

In [65]:
with open('after.txt', 'r', encoding='utf8') as f:
    code = f.read()
code = bytes(code, "utf8", errors='ignore')
tree = parser.parse(code)

In [63]:
print(tree.root_node)

(program (package_declaration (scoped_identifier scope: (scoped_identifier scope: (scoped_identifier scope: (identifier) name: (identifier)) name: (identifier)) name: (identifier))) (import_declaration (scoped_identifier scope: (scoped_identifier scope: (scoped_identifier scope: (identifier) name: (identifier)) name: (identifier)) name: (identifier))) (import_declaration (scoped_identifier scope: (scoped_identifier scope: (scoped_identifier scope: (identifier) name: (identifier)) name: (identifier)) name: (identifier))) (import_declaration (scoped_identifier scope: (scoped_identifier scope: (scoped_identifier scope: (identifier) name: (identifier)) name: (identifier)) name: (identifier))) (import_declaration (scoped_identifier scope: (scoped_identifier scope: (scoped_identifier scope: (identifier) name: (identifier)) name: (identifier)) name: (identifier))) (import_declaration (scoped_identifier scope: (scoped_identifier scope: (scoped_identifier scope: (scoped_identifier scope: (ident

In [69]:
def extract_node_info(node, ast_txt):
    ast_txt += '('+node.type
    if len(node.children) == 0:
        ast_txt += ', '+node.text.decode("utf8", errors='ignore')
    # ast_txt += '\n'
    for child in node.children:
        ast_txt = extract_node_info(child, ast_txt)
    ast_txt += ')'
    return ast_txt

def dfs(node, method_list, class_name): # Depth-First Search「深さ優先探索」
    if node.type == 'class_declaration':
        for child in node.children:
            if child.type == 'identifier':
                class_name = child.text.decode("utf8", errors='ignore')
    if node.type == 'method_declaration':
        method_dict = {}
        method_dict['class_name'] = class_name
        method_name = ''
        for child in node.children:
            if child.type == 'identifier' or child.type == 'formal_parameters':
                method_name += child.text.decode("utf8", errors='ignore')
        method_dict['method_name'] = method_name
        method_dict['content'] = node.text.decode("utf8", errors='ignore')
        ast_txt = ''
        method_dict['ast'] = extract_node_info(node, ast_txt)
        method_dict['position'] = [node.start_byte, node.end_byte]
        method_list.append(method_dict)
        return
    
    [dfs(child, method_list, class_name) for child in node.children]

In [70]:
method_list = []
class_name = ''
dfs(tree.root_node, method_list, class_name)

In [71]:
method_list

[{'class_name': 'RouteBuilder',
  'method_name': 'configure()',
  'content': 'public abstract void configure();',
  'ast': '(method_declaration(modifiers(public, public)(abstract, abstract))(void_type, void)(identifier, configure)(formal_parameters((, ()(), )))(;, ;))',
  'position': [846, 879]},
 {'class_name': 'RouteBuilder',
  'method_name': 'endpoint(String uri)',
  'content': 'public Endpoint<E> endpoint(String uri) {\n        return getEndpointResolver().resolve(uri);\n    }',
  'ast': '(method_declaration(modifiers(public, public))(generic_type(type_identifier, Endpoint)(type_arguments(<, <)(type_identifier, E)(>, >)))(identifier, endpoint)(formal_parameters((, ()(formal_parameter(type_identifier, String)(identifier, uri))(), )))(block({, {)(return_statement(return, return)(method_invocation(method_invocation(identifier, getEndpointResolver)(argument_list((, ()(), ))))(., .)(identifier, resolve)(argument_list((, ()(identifier, uri)(), ))))(;, ;))(}, })))',
  'position': [884, 98

In [7]:
for method in method_list:
    print(method['ast'])

(method_declaration(modifiers(public, public)(static, static))(type_parameters(<, <)(type_parameter(type_identifier, E))(>, >))(generic_type(type_identifier, Predicate)(type_arguments(<, <)(type_identifier, E)(>, >)))(identifier, and)(formal_parameters((, ()(formal_parameter(modifiers(final, final))(generic_type(type_identifier, Predicate)(type_arguments(<, <)(type_identifier, E)(>, >)))(identifier, left))(,, ,)(formal_parameter(modifiers(final, final))(generic_type(type_identifier, Predicate)(type_arguments(<, <)(type_identifier, E)(>, >)))(identifier, right))(), )))(block({, {)(expression_statement(method_invocation(identifier, ObjectHelper)(., .)(identifier, notNull)(argument_list((, ()(identifier, left)(,, ,)(string_literal(", ")(string_fragment, left)(", "))(), ))))(;, ;))(expression_statement(method_invocation(identifier, ObjectHelper)(., .)(identifier, notNull)(argument_list((, ()(identifier, right)(,, ,)(string_literal(", ")(string_fragment, right)(", "))(), ))))(;, ;))(return_

In [33]:
method_list[0]['position'][1][0]

15

In [17]:
code = bytes(method_list[0]['content'], "utf8", errors='ignore')
tree = parser.parse(code)